In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
mnist = input_data.read_data_sets('../../dataset/MNIST', one_hot=True)
Z_dim = 128
batch_size = 64
final_depth = 32
img_size = 32

# define inputs
X = tf.placeholder(tf.float32, shape= [None, 28, 28,1])
Z = tf.placeholder(tf.float32, shape= [None, Z_dim])
phase_train = tf.placeholder(tf.bool, name='phase_train')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataset/MNIST\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../dataset/MNIST\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../dataset/MNIST\t10k-images-idx3-ubyte.gz
Extracting ../../dataset/MNIST\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(32, 32), cmap='Greys_r')
    return fig

def batch_norm(x, train_phase, reuse, name='bn_layer'):
    #with tf.variable_scope(name) as scope:
    batch_norm = tf.layers.batch_normalization(
            inputs=x,
            momentum=0.9, epsilon=1e-5,
            center=True, scale=True,
            training = train_phase,
            reuse=reuse,
            name=name
    )
    return batch_norm

def conv2d(inputs, depth, reuse, name="conv"):
    Layer = tf.layers.conv2d(inputs,
                            depth,
                            [5,5],
                            strides=(2,2),
                            padding ='same',
                            kernel_initializer=tf.glorot_normal_initializer(),
                            reuse=reuse,
                            name=name)
    return Layer

def conv2d_transpose(inputs, depth, name = "conv_trnaspose"):
    Layer = tf.layers.conv2d_transpose(inputs,
                                       filters=depth,
                                       kernel_size=(5,5),
                                       strides=(2,2),
                                       padding='same',
                                       kernel_initializer=tf.glorot_normal_initializer(),
                                       name=name)
    return Layer


def generator(z, phase_train, name="generator"):
    with tf.variable_scope("generator"):        
        #inputs = tf.concat(values=[z, y], axis=1)
        inputs = z        
        # project and reshape
        h0 = tf.layers.dense(inputs,
                             int(img_size/16)*int(img_size/16)*final_depth*8,
                             kernel_initializer=tf.glorot_normal_initializer(),
                             name="project")
        h0_reshape = tf.reshape(h0, [-1,int(img_size/16),int(img_size/16),final_depth*8])       
        h0_BA = tf.nn.relu(batch_norm(h0_reshape, phase_train, None, name = "bn0"))        
        print(name," h0_BA ",h0_BA)
        
        h1 = conv2d_transpose(h0_BA,final_depth*4, name = "h1")
        h1_BA = tf.nn.relu(batch_norm(h1, phase_train, None, name = "bn1"))
        print(name," h1_BA ",h1_BA)
        
        h2 = conv2d_transpose(h1_BA,final_depth*2, name = "h2")
        h2_BA = tf.nn.relu(batch_norm(h2, phase_train, None, name = "bn2"))
        print(name," h2_BA ",h2_BA)
        
        h3 = conv2d_transpose(h2_BA,final_depth, name = "h3")
        h3_BA = tf.nn.relu(batch_norm(h3, phase_train, None, name = "bn3"))        
        print(name," h3_BA ",h3_BA)
        
        h4 = conv2d_transpose(h3_BA,1, name = "h4")
        h4_A = tf.nn.sigmoid(h4)
        print(name," h4_A ",h4_A)      
        return h4_A

def discriminator(x, phase_train, batch_size = 64, reuse=None, name="discriminator"):
    with tf.variable_scope("discriminator"):
        
        x_resize = tf.image.resize_images(x, [img_size,img_size],method=tf.image.ResizeMethod.BILINEAR,align_corners=False)
        #y_map = tf.tile(tf.reshape(y, [batch_size,1,1,10]), [1,img_size,img_size,1])
        
        #inputs = tf.concat(values=[x_resize, y_map], axis=3)
        inputs = x_resize
        
        h0 = conv2d(inputs, final_depth, reuse=reuse, name="h0")
        h0_A = tf.nn.leaky_relu(h0)
        print(name," h0_A ",h0_A)
        
        h1 = conv2d(h0_A, final_depth*2, reuse=reuse, name="h1")
        h1_BA = tf.nn.leaky_relu(batch_norm(h1, phase_train, reuse, name = "bn1"))
        print(name," h1_BA ",h1_BA)
        
        h2 = conv2d(h1_BA, final_depth*4, reuse=reuse, name="h2")
        h2_BA = tf.nn.leaky_relu(batch_norm(h2, phase_train, reuse, name = "bn2"))
        print(name," h2_BA ",h2_BA)
        
        h3 = conv2d(h2_BA, final_depth*8, reuse=reuse, name="h3")
        h3_BA = tf.nn.leaky_relu(batch_norm(h3, phase_train, reuse, name = "bn3"))
        print(name," h3_BA ",h3_BA)
        
        h4 = tf.layers.dense(tf.layers.flatten(h3_BA),
                             1,
                             kernel_initializer=tf.glorot_normal_initializer(),
                             reuse=reuse,
                             name="h4")
        print(name," h4 ",h4)
        
        return tf.nn.sigmoid(h4), h4

def loss(D_logit, D, D_logit_, D_):
    # D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
    # G_loss = -tf.reduce_mean(tf.log(D_fake))
    
    # Alternative losses:
    # -------------------
    D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit, labels=tf.ones_like(D)))
    D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_, labels=tf.zeros_like(D_)))
    D_loss = D_loss_real + D_loss_fake
    G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_, labels=tf.ones_like(D_)))
    
    return G_loss, D_loss

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [4]:
G_sample = generator(Z, phase_train)
D_real, D_logit_real = discriminator(X, phase_train, batch_size=batch_size, reuse=None)
D_fake, D_logit_fake = discriminator(G_sample, phase_train, batch_size=batch_size, reuse=True)

generator  h0_BA  Tensor("generator/Relu:0", shape=(?, 2, 2, 256), dtype=float32)
generator  h1_BA  Tensor("generator/Relu_1:0", shape=(?, 4, 4, 128), dtype=float32)
generator  h2_BA  Tensor("generator/Relu_2:0", shape=(?, 8, 8, 64), dtype=float32)
generator  h3_BA  Tensor("generator/Relu_3:0", shape=(?, 16, 16, 32), dtype=float32)
generator  h4_A  Tensor("generator/Sigmoid:0", shape=(?, 32, 32, 1), dtype=float32)
discriminator  h0_A  Tensor("discriminator/LeakyRelu/Maximum:0", shape=(?, 16, 16, 32), dtype=float32)
discriminator  h1_BA  Tensor("discriminator/LeakyRelu_1/Maximum:0", shape=(?, 8, 8, 64), dtype=float32)
discriminator  h2_BA  Tensor("discriminator/LeakyRelu_2/Maximum:0", shape=(?, 4, 4, 128), dtype=float32)
discriminator  h3_BA  Tensor("discriminator/LeakyRelu_3/Maximum:0", shape=(?, 2, 2, 256), dtype=float32)
discriminator  h4  Tensor("discriminator/h4/BiasAdd:0", shape=(?, 1), dtype=float32)
discriminator  h0_A  Tensor("discriminator_1/LeakyRelu/Maximum:0", shape=(?, 16,

In [5]:
t_vars = tf.trainable_variables()
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
g_vars = [var for var in t_vars if var.name.startswith("generator")]
d_vars = [var for var in t_vars if var.name.startswith("discriminator")]

In [6]:
G_loss, D_loss = loss(D_logit_real, D_real, D_logit_fake, D_fake)
D_solver = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(D_loss, var_list=d_vars)
G_solver = tf.train.AdamOptimizer(0.0002, beta1=0.5).minimize(G_loss, var_list=g_vars)

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if not os.path.exists('out/'):
    os.makedirs('out/')
    
for it in range(50000):
    X_mb, _ = mnist.train.next_batch(batch_size)
    X_mb = np.reshape(X_mb, [batch_size,28 ,28, 1])
    _, D_loss_curr, _ = sess.run([D_solver, D_loss, update_ops], feed_dict={X: X_mb,
                                                                            Z: sample_Z(batch_size, Z_dim),
                                                                            phase_train: True})
    _, G_loss_curr,_ = sess.run([G_solver, G_loss, update_ops], feed_dict={X: X_mb,
                                                                           Z: sample_Z(batch_size, Z_dim),
                                                                           phase_train: True})

    if it % 1000 == 0:
        print('Iter: {}, D loss: {:.4}, G_loss: {:.4}'.format(it,D_loss_curr,G_loss_curr))
        
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim),
                                                phase_train: False})

        fig = plot(samples)
        plt.savefig('out/ex04_DCGAN/{}.png'.format(str(it).zfill(5)), bbox_inches='tight')
        plt.close(fig)
sess.close()

Iter: 0, D loss: 1.648, G_loss: 6.347
Iter: 1000, D loss: 0.1373, G_loss: 4.767
Iter: 2000, D loss: 0.02996, G_loss: 4.917
Iter: 3000, D loss: 0.1229, G_loss: 5.068
Iter: 4000, D loss: 0.2379, G_loss: 6.186
Iter: 5000, D loss: 0.3849, G_loss: 7.368
Iter: 6000, D loss: 0.09812, G_loss: 5.023
Iter: 7000, D loss: 0.2827, G_loss: 5.475
Iter: 8000, D loss: 0.09995, G_loss: 4.103
Iter: 9000, D loss: 0.04986, G_loss: 8.765
Iter: 10000, D loss: 0.08772, G_loss: 4.751
Iter: 11000, D loss: 0.02194, G_loss: 5.326
Iter: 12000, D loss: 0.06185, G_loss: 6.048
Iter: 13000, D loss: 0.08934, G_loss: 4.244
Iter: 14000, D loss: 0.3208, G_loss: 7.644
Iter: 15000, D loss: 1.762, G_loss: 13.35
Iter: 16000, D loss: 0.1075, G_loss: 4.358
Iter: 17000, D loss: 0.00981, G_loss: 5.85
Iter: 18000, D loss: 0.02155, G_loss: 5.588
Iter: 19000, D loss: 0.005004, G_loss: 7.064
Iter: 20000, D loss: 0.01946, G_loss: 5.391
Iter: 21000, D loss: 0.04836, G_loss: 5.32
Iter: 22000, D loss: 0.02307, G_loss: 6.773
Iter: 23000, 